In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from os import path
from pathlib import Path  

#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [2]:
# import files
data_file = path.join('/Users/louxuwen/Desktop/Documents/GitHub/SP23-BENG213/Project/Map Gene to KEGG Pathway/down_data_matrix.csv') # Data matrix
gene_file = path.join('/Users/louxuwen/Desktop/Documents/GitHub/SP23-BENG213/Project/Map Gene to KEGG Pathway/Homo_sapiens.tsv') # Gene Symbol and ID
pathway_file = path.join('/Users/louxuwen/Desktop/Documents/GitHub/SP23-BENG213/Project/Map Gene to KEGG Pathway/Pathway_Down.csv') # all the pathways used in the heatmap
gene_to_pathway_file = path.join('/Users/louxuwen/Desktop/Documents/GitHub/SP23-BENG213/Project/Map Gene to KEGG Pathway/Gene_to_Pathway.csv') # every pathway and the relevant genes

# dataframe of metadata
data = pd.read_csv(data_file,index_col=0)
gene_info = pd.read_csv(gene_file, sep="\t")
pathway = pd.read_csv(pathway_file,index_col=0).fillna("0")
gene_to_pathway = pd.read_csv(gene_to_pathway_file)

In [7]:
gene_to_pathway

,Pathway,Gene
0,hsa00010,10327
1,hsa00010,124
2,hsa00010,125
3,hsa00010,126
4,hsa00010,127
...,...,...
36117,hsa05418,91860
36118,hsa05418,92
36119,hsa05418,93
36120,hsa05418,9446


In [4]:
gene_info

,#tax_id,GeneID,Symbol,LocusTag,Synonyms,dbXrefs,chromosome,map_location,description,type_of_gene,Symbol_from_nomenclature_authority,Full_name_from_nomenclature_authority,Nomenclature_status,Other_designations,Modification_date,Feature_type
0,9606,1,A1BG,-,A1B|ABG|GAB|HYST2477,MIM:138670|HGNC:HGNC:5|Ensembl:ENSG00000121410...,19,19q13.43,alpha-1-B glycoprotein,protein-coding,A1BG,alpha-1-B glycoprotein,O,alpha-1B-glycoprotein|HEL-S-163pA|epididymis s...,20230329,-
1,9606,2,A2M,-,A2MD|CPAMD5|FWP007|S863-7,MIM:103950|HGNC:HGNC:7|Ensembl:ENSG00000175899...,12,12p13.31,alpha-2-macroglobulin,protein-coding,A2M,alpha-2-macroglobulin,O,alpha-2-macroglobulin|C3 and PZP-like alpha-2-...,20230405,-
2,9606,3,A2MP1,-,A2MP,HGNC:HGNC:8|Ensembl:ENSG00000291190|AllianceGe...,12,12p13.31,alpha-2-macroglobulin pseudogene 1,pseudo,A2MP1,alpha-2-macroglobulin pseudogene 1,O,pregnancy-zone protein pseudogene,20230329,-
3,9606,9,NAT1,-,AAC1|MNAT|NAT-1|NATI,MIM:108345|HGNC:HGNC:7645|Ensembl:ENSG00000171...,8,8p22,N-acetyltransferase 1,protein-coding,NAT1,N-acetyltransferase 1,O,arylamine N-acetyltransferase 1|N-acetyltransf...,20230509,-
4,9606,10,NAT2,-,AAC2|NAT-2|PNAT,MIM:612182|HGNC:HGNC:7646|Ensembl:ENSG00000156...,8,8p22,N-acetyltransferase 2,protein-coding,NAT2,N-acetyltransferase 2,O,arylamine N-acetyltransferase 2|N-acetyltransf...,20230509,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169029,741158,8923215,trnD,-,-,-,MT,-,tRNA-Asp,tRNA,-,-,-,-,20200909,-
169030,741158,8923216,trnP,-,-,-,MT,-,tRNA-Pro,tRNA,-,-,-,-,20200909,-
169031,741158,8923217,trnA,-,-,-,MT,-,tRNA-Ala,tRNA,-,-,-,-,20200909,-
169032,741158,8923218,COX1,-,-,-,MT,-,cytochrome c oxidase subunit I,protein-coding,-,-,-,cytochrome c oxidase subunit I,20200909,-


In [ ]:
pathways = pathway.index.tolist()
pathway_name = []
pathway_number = []
for i in range(len(pathways)):
    temp = pathway.iloc[i]
    temp2 = []
    for j in range(4):
        if temp[j] != "0":
            pathway_name.append(pathways[i])
            pathway_number.append(temp[j])
pathway_number = set(pathway_number)            

In [ ]:
# get gene IDs
# note that 7 percent (~252/3418 genes does not have matching symbol=ID)
Gene_IDs = []
for i in range(len(data.index.tolist())):
    Gene_name = data.iloc[i][0]
    
    # get gene symbol
    if "," in Gene_name:
        temp = Gene_name.split(",")
        if temp[0] != "":
            Gene_name = temp[0]
        else:
            Gene_name = temp[1].lstrip()
    
    # gene_symbol to ID
    ID = gene_info.loc[gene_info['Symbol'] == Gene_name]['GeneID'].tolist()
    if ID == []:
        Gene_IDs.append("None")
    else:
        Gene_IDs.append(ID[0])

In [ ]:
# add the IDs column in and drop the genes that doesn't have an ID
data["ID"] = Gene_IDs
data = data[data['ID'] != "None"]

In [ ]:
# get the pathway
# only 1402/3166 genes belong to a functional pathway
pathways_by_gene = []
pathways_by_gene_column = []
for i in range(len(data.index.tolist())):
    temp_ID = data.iloc[i][28]
    
    # get corresponding pathways
    temp_pathways = gene_to_pathway[gene_to_pathway['Gene']== data.iloc[i][28]]["Pathway"].tolist()
    
    if len(temp_pathways) == 0:
        pathways_by_gene_column.append("none")
    else:
        pathways_by_gene.append(temp_pathways)
        pathways_by_gene_column.append(" ".join(temp_pathways))

In [ ]:
data["All Pathways"] = pathways_by_gene_column
data = data[data['All Pathways'] != "none"]

## output the data file as long as the gene is involved with a pathways ##

In [ ]:
filepath = Path('/Users/louxuwen/Desktop/Documents/GitHub/SP23-BENG213/Project/Map Gene to KEGG Pathway/Data_All_Pathways.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
data.to_csv(filepath)  

## continue to shrink data only if a gene is involved with a pathway actually used in the heatmap ##

In [ ]:
# find the pathway that's involved with the heat map
pathways_involved = []
pathways_involved_column = []
for i in range(len(data.index.tolist())):
    a = pathways_by_gene[i]
    temp = []

    for j in a:
        if j in pathway_number:
            temp.append(j)
    
    if len(temp) == 0:
        pathways_involved_column.append("none")
    else:
        pathways_involved.append(temp)
        pathways_involved_column.append(" ".join(temp))

In [ ]:
data["Involved Pathways"] = pathways_involved_column
data = data[data['Involved Pathways'] != "none"]

In [ ]:
drugs = data.columns.values.tolist()[1:28]
functions = pathway.index.tolist()

In [ ]:
# create the finalmatrix
final_matrix = [[0.0] * len(drugs)]*len(functions)
counter=0
for i in range(len(data.index.tolist())):
    temp_pathways = pathways_involved[i]
    temp_values = data.iloc[i].tolist()[1:28]
    temp_functions = []
    for j in temp_pathways:
        a = pathway[pathway["pathway_1"] == j].index.tolist()
        if len(a)>0:
            temp_functions.append(a[0])
        else:
            a = pathway[pathway["pathway_2"] == j].index.tolist()
            if len(a)>0:
                temp_functions.append(a[0])
            else:
                a = pathway[pathway["pathway_3"] == j].index.tolist()
                if len(a)>0:
                    temp_functions.append(a[0])
                else:
                    a = pathway[pathway["pathway_4"] == j].index.tolist()
                    if len(a)>0:
                        temp_functions.append(a[0])
    for k in temp_functions:
        counter = counter + 1
        temp_index = functions.index(k)
        final_matrix[temp_index] = [final_matrix[temp_index][n] + temp_values[n] for n in range(len(temp_values))] 


In [ ]:
final_data_functions_by_drugs = pd.DataFrame(final_matrix,index = functions, columns=drugs)
final_data_functions_by_drugs

In [ ]:
filepath = Path('/Users/louxuwen/Desktop/Documents/GitHub/SP23-BENG213/Project/Map Gene to KEGG Pathway/Final_data_functions_by_drugs.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
final_data_functions_by_drugs.to_csv(filepath)  

## establish matrix as pathway by drugs ##